In [1]:
import sqlite3

In [2]:
# 프로젝트 루트 디렉터리의 db.sqlite3 파일에 연결
conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

In [3]:
# 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name TEXT NOT NULL,
    quantity INTEGER NOT NULL,
    price INTEGER NOT NULL
)
''')


In [5]:
# 상품 정보를 데이터베이스에 삽입하는 함수
def insert_product(product_name, quantity, price):
    # 가격에서 콤마 제거 및 정수 변환
    price = int(price.replace(',', ''))
    cursor.execute('INSERT INTO products (product_name, quantity, price) VALUES (?, ?, ?)', 
                   (product_name, quantity, price))
    conn.commit()

In [6]:
# 상품 정보 삽입 예시
insert_product('떡케익5호', 1, '54,000')
insert_product('무지개 백설기 케익', 1, '51,500')
insert_product('미니 백설기', 35, '31,500')
insert_product('개별 모듬팩', 1, '13,500')

In [7]:
# 데이터베이스에서 상품 정보 조회
cursor.execute('SELECT * FROM products')
rows = cursor.fetchall()
rows

[(1, '떡케익5호', 1, 54000),
 (2, '무지개 백설기 케익', 1, 51500),
 (3, '미니 백설기', 35, 31500),
 (4, '개별 모듬팩', 1, 13500)]

In [8]:
for row in rows:
    print(row)

(1, '떡케익5호', 1, 54000)
(2, '무지개 백설기 케익', 1, 51500)
(3, '미니 백설기', 35, 31500)
(4, '개별 모듬팩', 1, 13500)


In [9]:
# 데이터베이스 연결 종료
conn.close()

종료 후 다시 접속해 확인

In [10]:
# 프로젝트 루트 디렉터리의 db.sqlite3 파일에 연결
conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

cursor.execute('SELECT * FROM products')
rows = cursor.fetchall()

for row in rows:
    print(row)

(1, '떡케익5호', 1, 54000)
(2, '무지개 백설기 케익', 1, 51500)
(3, '미니 백설기', 35, 31500)
(4, '개별 모듬팩', 1, 13500)


In [11]:
conn.close()

# langchain

In [4]:
from langchain_community.utilities import SQLDatabase

# Assign the 'db.sqlite3' in your project directory to the 'db' variable using the SQLDatabase class
db = SQLDatabase.from_uri("sqlite:///db.sqlite3")
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['products']


In [14]:
# Optionally, run a test query to ensure everything is set up correctly
test_query_result = db.run("SELECT * FROM products LIMIT 10;")
print(test_query_result)

[(1, '떡케익5호', 1, 54000), (2, '무지개 백설기 케익', 1, 51500), (3, '미니 백설기', 35, 31500), (4, '개별 모듬팩', 1, 13500)]


## Convert question to SQL query

In [1]:
import os

os.environ["OPENAI_API_KEY"]

KeyError: 'OPENAI_API_KEY'

In [ ]:
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"]

In [5]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many products are there"})
response

'SELECT COUNT(*) AS total_products FROM products;'

In [6]:
db.run(response)

'[(4,)]'

In [8]:
chain.get_prompts()

[PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use date(\'now\') function to get the current date, if the question i

In [7]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

# Django 

In [9]:
from django.db import connection

with connection.cursor() as cursor:
    cursor.execute("SELECT username FROM auth_user")
    for row in cursor.fetchall():
        print(row)

ImproperlyConfigured: Requested setting DATABASES, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.